In [128]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [129]:
df = pd.read_csv("insuranceData.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [130]:
# Creating training and testing dataset

y = df.iloc[:,-1]
X = df.iloc[:,:-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [131]:
# Scaling features

scaler = MinMaxScaler()
xTrainScaled = scaler.fit_transform(X_train)
xTestScaled = scaler.transform(X_test)

In [132]:
xTrainScaled[0:5]

array([[0.09090909, 1.        ],
       [0.90909091, 1.        ],
       [0.70454545, 1.        ],
       [0.84090909, 0.        ],
       [0.06818182, 1.        ]])

In [133]:
ytrain = y_train.to_numpy()
ytrain = ytrain.reshape((-1,1))

 ## Neural network with 1 hidden layer and adjustable number of nerons
 ### Activation functions: Sigmoid for output layer and ReLU for 

In [134]:
# define the ReLU function
def relu(x,derivative=False):
    if (derivative):
        return np.where(x > 0, 1, 0)
    else:
        return np.maximum(x,0)
    
    
    
    
# define the sigmoid function 
def sigmoid(x):
    return(1/(1+(np.exp(-x))))



# define the loss function
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*(np.log(1-y_predicted_new)))


In [135]:
class annModel:
    def __init__(self):
        self.hiddenWeights = None
        self.outputWeights = None
    
    def fit(self,X,y,epochs,numHidden):
        self.hiddenWeights, self.outputWeights = self.gradientDescent(X,y,epochs,numHidden)
    
    def predict(self,xtest):
        inputLayer_outputs = np.hstack((xtest, np.ones((xtest.shape[0], 1))))
        hiddenLayer_outputs = np.hstack((relu(np.dot(inputLayer_outputs, self.hiddenWeights)), np.ones((xtest.shape[0], 1))))
        predictions = sigmoid(np.dot(hiddenLayer_outputs, self.outputWeights))
        return predictions
        
    
    def gradientDescent(self,X,y,epochs,numHidden):
        
        # learning rate
        alpha = 0.01
        
        
        # initializing weights
        # the +1 in the 1st dimension of the weight matrices is for the bias weight
        hiddenWeights = np.random.random((X.shape[1] + 1, numHidden))
        outputWeights = np.random.random((numHidden + 1, 1))
        
        for i in range (epochs):
            
            # Forward Propagation
            # we stack a column of ones along with input X to take care of biases
            inputLayer_outputs = np.hstack((X, np.ones((X.shape[0], 1))))
            hiddenLayer_outputs = np.hstack((relu(np.dot(inputLayer_outputs, hiddenWeights)), np.ones((X.shape[0], 1))))
            outputLayer_outputs = sigmoid(np.dot(hiddenLayer_outputs, outputWeights))
            loss = log_loss(y,outputLayer_outputs)
            
            # Error terms for output layer = loss' x sigmoid'(a) 
            outputError = (outputLayer_outputs - y)  
            hiddenError = relu(hiddenLayer_outputs,derivative=True)[:,:-1]*(np.dot(outputError,outputWeights.T[:,:-1]))
            
            
            # partial derivatives
            hidden_pd = inputLayer_outputs[:, :, np.newaxis] * hiddenError[: , np.newaxis, :]
            output_pd = hiddenLayer_outputs[:, :, np.newaxis] * outputError[:, np.newaxis, :]

            # average for total gradients
            total_hidden_gradient = np.average(hidden_pd, axis=0)
            total_output_gradient = np.average(output_pd, axis=0)

            # update weights
            hiddenWeights += - alpha * total_hidden_gradient
            outputWeights += - alpha * total_output_gradient
            
            if i%100 == 0:
                print(f'Epoch: {i}  Loss: {loss}')
            
            if (i==epochs-1):
                print(f'Epoch: {i}  Loss: {loss}')
                return hiddenWeights, outputWeights
        
    

In [136]:
model = annModel()
model.fit(xTrainScaled,ytrain,2000,10)

Epoch: 0  Loss: 2.0175934813828618
Epoch: 100  Loss: 0.6529351343874926
Epoch: 200  Loss: 0.6158858930263048
Epoch: 300  Loss: 0.5870281655458224
Epoch: 400  Loss: 0.5603386119990362
Epoch: 500  Loss: 0.5355579050153412
Epoch: 600  Loss: 0.5126352046567336
Epoch: 700  Loss: 0.49165152680301644
Epoch: 800  Loss: 0.4726740592538317
Epoch: 900  Loss: 0.455706404817764
Epoch: 1000  Loss: 0.4406801645631713
Epoch: 1100  Loss: 0.4274784187495327
Epoch: 1200  Loss: 0.41594714896607327
Epoch: 1300  Loss: 0.4059097749804055
Epoch: 1400  Loss: 0.3971813826003627
Epoch: 1500  Loss: 0.38958029315414744
Epoch: 1600  Loss: 0.38293617634954624
Epoch: 1700  Loss: 0.37709487822994203
Epoch: 1800  Loss: 0.37192060011821243
Epoch: 1900  Loss: 0.36729618304585904
Epoch: 1999  Loss: 0.3631619778841662


In [137]:
model.predict(xTestScaled)

array([[0.50356293],
       [0.86465945],
       [0.85305959],
       [0.68571586],
       [0.57998547],
       [0.21334935]])

In [138]:
y_test

3     0
5     1
15    1
7     1
22    1
26    0
Name: bought_insurance, dtype: int64